In [40]:
import pandas as pd
from pprint import PrettyPrinter
from pathlib import Path
import os
import requests

In [41]:
""" Get stuff out of Netfile v2 API
"""
from pprint import PrettyPrinter
from pathlib import Path
import os
import requests

BASE_URL = 'https://netfile.com/api/campaign'
CONTRIBUTION_FORM = 'F460A'
EXPENDITURE_FORM = 'F460E'

PARAMS = { 'aid': 'COAK' }

def get_auth_from_env_file(filename: str='.env'):
    """ Split .env file on newline and look for API_KEY and API_SECRET
        Return their values as a tuple
    """
    env_file=Path(filename)
    auth_keys = [ 'API_KEY', 'API_SECRET' ]
    if env_file.exists():
        auth = tuple( v for _, v in sorted([
            ln.split('=') for ln in
            env_file.read_text(encoding='utf8').strip().split('\n')
            if ln.startswith(auth_keys[0]) or ln.startswith(auth_keys[1])
        ], key=lambda ln: auth_keys.index(ln[0])))
    else:
        auth=tuple(os.environ[key] for key in auth_keys)
            
    return auth

NET=get_auth_from_env_file()

pp = PrettyPrinter()

In [42]:
def get_auth_from_env_file(filename: str='.env'):
    """ Split .env file on newline and look for API_KEY and API_SECRET
        Return their values as a tuple
    """
    env_file=Path(filename)
    auth_keys = [ 'OAKDATA_KEY', 'OAKDATA_SECRET' ]
    if env_file.exists():
        auth = tuple( v for _, v in sorted([
            ln.split('=') for ln in
            env_file.read_text(encoding='utf8').strip().split('\n')
            if ln.startswith(auth_keys[0]) or ln.startswith(auth_keys[1])
        ], key=lambda ln: auth_keys.index(ln[0])))
    else:
        auth=tuple(os.environ[key] for key in auth_keys)
            
    return auth
def list_filers():
    """ Get all the elections
    """
    url = f'{BASE_URL}/filer/v101/filers?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=NET)
    if res.status_code == 500:
        print('ping')
        return list_filers()
    else:
        body = res.json()
        return body['results']

AUTH=get_auth_from_env_file()


In [43]:
res = requests.get(url='https://data.oaklandca.gov/resource/v93t-prrc.json', auth=AUTH)

In [44]:
candidate_table=pd.DataFrame(res.json())

In [45]:
template = pd.read_csv('input/filer_to_candidate.csv')

In [46]:
template.columns

Index(['filer_name', 'is_terminated', 'sos_id', 'type', 'local_agency_id',
       'election_year', 'candidate', 'contest', 'citywide', 'incumbent',
       'start', 'end', 'is_winner', 'ballot_status'],
      dtype='object')

In [50]:
candidate_table[['filer_name','is_terminated','fppc_id','filer_type','election_year','candidate_name','office_name']].iloc[650]

filer_name                                       NaN
is_terminated                                    NaN
fppc_id                                          NaN
filer_type                                       NaN
election_year                                   2010
candidate_name                       Sinclair, Craig
office_name       City Council - City of Oakland - 4
Name: 650, dtype: object

In [48]:
filers_response=list_filers()
filers_response
table = [
    {
        'filerNid': item['filerNid'],
        'Filer Name':item['filerName'],
        'committeeName':item.get('electionInfluences', [None])[0]['committeeName'] if item.get('electionInfluences', None) else None,
        'isTerminated': item.get('isTerminated', {}),
        # Get the latest status and conditons for epmty lists to avoid indexError
        'fppc_id':item.get('registrations',{}).get('CA SOS',None),
        'Filer Type': item['committeeTypes'][0],
        'election_year': item.get('electionInfluences', [None])[0]['electionDate'][:4] if item.get('electionInfluences', None) else None,
        'candidateName': item['candidateName'],
        'measure':f"Measure {item.get('electionInfluences', [None])[0]['measure']['measureNumber']}" if item.get('electionInfluences', None) and item.get('electionInfluences', [None])[0].get('measure', None) else None,
        'seat':item.get('electionInfluences', [None])[0]['seat']['officeName'] if item.get('electionInfluences', None) and item.get('electionInfluences', [None])[0].get('seat', None) else None,
    } 
    for item in filers_response if item['committeeTypes'] in [['Person'],['Candidate or Officeholder'], ['Primarily Formed Measure'],['Primarily Formed Candidate']]
]
df = pd.DataFrame(table)

In [49]:
df

,filerNid,Filer Name,committeeName,isTerminated,fppc_id,Filer Type,election_year,candidateName,measure,seat
0,210906813,Tariq Ikharo for Oakland City Council 2024,"Ikharo, Tariq",False,Pending,Candidate or Officeholder,2024,"Ikharo, Tariq",None,City Council - City of Oakland - 3
1,211048114,"Garrett, Lere","Garrett, Lere",False,None,Person,2024,"Garrett, Lere",None,City Council Member At-Large - City of Oakland
2,210526254,Wang for Oakland City Council 2024,"Wang, Charlene",False,1467592,Candidate or Officeholder,2024,"Wang, Charlene",None,City Council Member At-Large - City of Oakland
3,208539515,Goolsby City Council District 7 2024,"Goolsby, Merika",False,None,Candidate or Officeholder,2024,"Goolsby, Merika",None,City Council - City of Oakland - 7
4,210965594,Coalition to Reclaim Oakland's Committee to Re...,Coalition to Reclaim Oakland's Committee to Re...,False,1467194,Primarily Formed Measure,2024,None,Measure TBD 2,None
...,...,...,...,...,...,...,...,...,...,...
428,121706546,McCullough For City Council,None,True,1304061,Candidate or Officeholder,None,"McCullough, Patrick",None,None
429,121706516,Jane Brunner For City Council 2008,None,True,1245211,Candidate or Officeholder,None,None,None,None
430,121706486,Neighbors For Russo,None,True,931297,Candidate or Officeholder,None,"Russo, John",None,None
431,121706456,Kaplan For Oakland,None,True,1303541,Candidate or Officeholder,None,"Kaplan, Rebecca",None,None
